In [1]:
! pip install langchain
! pip install langchain_openai
! pip install langchain_community
! pip install pymongo
! pip install python-dotenv
! pip install azure-core
! pip install azure-cosmos


[notice] A new release of pip is available: 23.3.1 -> 24.0
[notice] To update, run: C:\Users\Khelan Modi\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip



[notice] A new release of pip is available: 23.3.1 -> 24.0
[notice] To update, run: C:\Users\Khelan Modi\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip



[notice] A new release of pip is available: 23.3.1 -> 24.0
[notice] To update, run: C:\Users\Khelan Modi\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip



[notice] A new release of pip is available: 23.3.1 -> 24.0
[notice] To update, run: C:\Users\Khelan Modi\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip



[notice] A new release of pip is available: 23.3.1 -> 24.0
[notice] To update, run: C:\Users\Khelan Modi\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip



[notice] A new release of pip is available: 23.3.1 -> 24.0
[notice] To update, run: C:\Users\Khelan Modi\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip



[notice] A new release of pip is available: 23.3.1 -> 24.0
[notice] To update, run: C:\Users\Khelan Modi\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


In [16]:
from dotenv import dotenv_values
import gradio as gr
from langchain.chains import ConversationalRetrievalChain
from langchain.globals import set_llm_cache
from langchain.memory import ConversationBufferMemory
from langchain.prompts import PromptTemplate
from langchain_community.cache import AzureCosmosDBSemanticCache
from langchain_community.chat_message_histories import MongoDBChatMessageHistory
from langchain_community.vectorstores.azure_cosmos_db import (
    AzureCosmosDBVectorSearch,
    CosmosDBSimilarityType,
    CosmosDBVectorSearchType)
from langchain_openai import AzureChatOpenAI, AzureOpenAIEmbeddings
import pymongo

In [19]:
import json
import openai
import pymongo

from dotenv import dotenv_values
from openai import AzureOpenAI

env_name = "example.env" 
config = dotenv_values(env_name)

openai.api_type = config['openai_type']
# Azure OpenAI connection details
openai_endpoint = config['openai_api_endpoint']
openai_key = config['openai_api_key']
openai_version = config['openai_api_version']
openai_embeddings_deployment = config['openai_embeddings_deployment']
openai_embeddings_model = config['openai_api_version']
openai_embeddings_dimensions = int(config['openai_embeddings_dimensions'])
openai_completions_deployment = config['openai_completions_deployment']
#openai_completions_model = config['openai_completions_model']


In [3]:
# Connection string
mongo_conn = config['mongo_vcore_connection_string']
mongo_client = pymongo.MongoClient(mongo_conn)

# Database name
DATABASE_NAME = "ResturantChain"
db = mongo_client[DATABASE_NAME]

# Drop the database if it already exists (consider if this is really needed, as it will remove all existing data)
#mongo_client.drop_database(DATABASE_NAME)

# Collection names
collection_names = ["Contoso", "Contoso West", "Users"]

# Iterate through the collection names and create them if they do not exist
for collection_name in collection_names:
    if collection_name not in db.list_collection_names():
        # Creates a collection
        db.create_collection(collection_name)
        print(f"Created collection '{collection_name}'.")
    else:
        print(f"Using existing collection: '{collection_name}'.")

Using existing collection: 'Contoso'.
Using existing collection: 'Contoso West'.
Using existing collection: 'Users'.


In [4]:
db.command({
  'createIndexes': 'FoodCollection',
  'indexes': [
    {
      'name': 'vectorSearchIndex',
      'key': {
        "Embedding": "cosmosSearch"
      },
      'cosmosSearchOptions': {
        'kind': 'vector-hnsw',
        'm': 16,
        'efConstruction': 40,
        'similarity': 'COS',
        'dimensions': 1536
      }
    }
  ]
});

In [9]:
import json 

data_file = open(file="./data/output.json", mode="r") 
data = json.load(data_file)
data_file.close()
collection = db['FoodCollection']
result = collection.insert_many(data)

print(f"Number of data points added: {len(result.inserted_ids)} in {collection.name}")

UnicodeDecodeError: 'charmap' codec can't decode byte 0x81 in position 18432: character maps to <undefined>

In [21]:
from langchain.vectorstores.azure_cosmos_db import AzureCosmosDBVectorSearch
from langchain.core.prompts import ChatPromptTemplate
from langchain.openai import ChatOpenAI, OpenAIEmbeddings
from langchain.chains.combine_documents import createStuffDocumentsChain
from langchain.chains.retrieval import createRetrievalChain
from langchain.document_loaders.fs.text import TextLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter

ModuleNotFoundError: No module named 'langchain.core'

In [22]:
azure_openai_embeddings = AzureOpenAIEmbeddings(
    azure_deployment = openai_embeddings_deployment,
    api_key= openai_key,
    azure_endpoint= openai_endpoint,
    model= openai_embeddings_model,
    dimensions= openai_embeddings_dimensions,
)

azure_openai_embeddings.embed_query("Hello, this is a test!")

NotFoundError: Error code: 404 - {'error': {'code': 'DeploymentNotFound', 'message': 'The API deployment for this resource does not exist. If you created the deployment within the last 5 minutes, please wait a moment and try again.'}}

In [ ]:
cdb = AzureCosmosDBVectorSearch(
    collection= cosmos_collection,
    embedding=azure_openai_embeddings)

vectorstore = cdb.from_connection_string(
    connection_string=cosmos_conn,
    namespace = cosmos_database + "." + cosmos_collection,
    embedding = azure_openai_embeddings,
    embedding_key = cosmos_vector_property,
    text_key = "overview")

In [ ]:
vectorstore.similarity_search_with_score("Buzz Lightyear", k=5, score_threshold=0.2)

In [ ]:
prompt_template = """
You are an upbeat AI assistant who is excited to help answer questions. 
You can use this context

{context},

or this chat history

{chat_history},

to answer this question. 

Question: {question}
If you don't know the answer, just say that you don't know. Don't try to make up an answer.
"""
chatbot_prompt = PromptTemplate(
    template = prompt_template, input_variables = ["context", "question", "chat_history"])

In [ ]:
def prepare_chain():
    llm = AzureChatOpenAI(
            azure_endpoint = openai_endpoint,
            api_key = openai_key,
            api_version = openai_version,
            azure_deployment = "completions", 
            cache = True,
            n = 1)

    retriever = vectorstore.as_retriever(
    search_type = "similarity",
    search_kwargs = {"k": 5, 'score_threshold': 0.2})

    sem_qa = ConversationalRetrievalChain.from_llm(
    llm = llm,
    chain_type = "stuff",
    retriever = retriever,
    return_source_documents = True,
    combine_docs_chain_kwargs = {"prompt": chatbot_prompt})

    similarity_algorithm = CosmosDBSimilarityType.COS
    kind = CosmosDBVectorSearchType.VECTOR_IVF
    num_lists = 1
    score_threshold = 0.99

    sem_cache = AzureCosmosDBSemanticCache(
            cosmosdb_connection_string = cosmos_conn,
            cosmosdb_client = None,
            embedding = azure_openai_embeddings,
            database_name = cosmos_database, 
            collection_name = cosmos_semcache, 
            similarity = similarity_algorithm,
            num_lists = num_lists,
            kind = kind,
            dimensions = openai_embeddings_dimensions, 
            score_threshold = score_threshold)

    set_llm_cache(sem_cache)

    return retriever, llm, sem_qa, sem_cache 

In [ ]:
retriever, llm, chain, semantic_cache = prepare_chain()

In [ ]:
# Clearing Semantic Cache inbetween testing
cache.drop_indexes()
database.drop_collection(cache)

In [ ]:
chain.invoke({'question': "Tell me about movies with Buzz Lightyear.", 'chat_history': []})


In [ ]:
chain.invoke({'question': "Tell me something about films with Buzz Lightyear.", 'chat_history': []})


In [ ]:
chatbot_chain_retriever, chatbot_chain_llm, chatbot_chain, chatbot_chain_cache = prepare_chain()

# Clearing Semantic Cache inbetween testing
cache.drop_indexes()
database.drop_collection(cache)
database.drop_collection(chathistory)

cosmos_message_history = MongoDBChatMessageHistory(
    session_id = "test_session",
    connection_string = cosmos_conn,
    database_name = cosmos_database,
    collection_name = cosmos_chat_history)

conversational_memory = ConversationBufferMemory(
    chat_memory=cosmos_message_history,
    memory_key='chat_history',
    return_messages=True)

# Load history locally. Grab last 
hist = []
with gr.Blocks() as demo:
    chatbot = gr.Chatbot()
    msg = gr.Textbox()
    clear = gr.Button("Clear")

    def user(user_message, chat_history):
        # Get response from QA chain
        response = chatbot_chain.invoke({"question": user_message, "chat_history":conversational_memory.buffer_as_messages[-6:]},temperature=0.2)
        # Append user message and response to chat history
        hist.append(["User: "+user_message, "Chatbot: "+response['answer']])
        cosmos_message_history.add_user_message(user_message)
        cosmos_message_history.add_ai_message(response['answer'])
        return gr.update(value=""), hist

    msg.submit(user, [msg, chatbot], [msg, chatbot], queue=False)
    clear.click(lambda: None, None, chatbot, queue=False)

In [ ]:
demo.launch(debug=True)